In [1]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from src.utils.utils import *

# Extrair caracteristicas matriz GLCM

In [2]:
path_json = "D:\\mathe\\Documents\\PythonNotebooks\\Radiomica\\outputs\\mamografia\\extracted_metadata"

angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
distances = [1, 3]

### CMMD

In [3]:
name_json = "extracted_metadata_CMMD"

list_metadata = load_json(name_json, path_json)
glcm_features = []
labels = []

for metadata in list_metadata:
    metadata_csv = metadata['metadata_csv']
    
    tmp_path = "\\".join(metadata_csv['image_path'][0].split("\\")[:-1])  # noqa: E501
    directory = Path(tmp_path)
    paths_dicom_file = list(directory.rglob("*.dcm*"))
    
    for path_dicom_file in paths_dicom_file:
        dicom_file = dcmread(path_dicom_file)
        
        labels.append(metadata_csv['classification'])
        
        # Carrega a matriz da imagem original
        image = dicom_file.pixel_array

        # Redimensiona a imagem original e salva no array
        if image.max() > 256:
            image = (image / 256).astype(np.uint8)

        # image = cv2.equalizeHist(image)
        features = get_glcm_features(image, distances, angles, 256, True, True, properties)
        glcm_features.append(features)

columns = []
angles_labels = get_angles_labels(angles)

for name in properties:
    for distance in distances:
        for ang in angles_labels:
            columns.append(name + "_ang_" + ang + "_dist_" + str(distance))

glcm_df_CMMD = pd.DataFrame(glcm_features, columns=columns)
glcm_df_CMMD['pathology'] = labels
glcm_df_CMMD.to_csv("../../outputs/mamografia/matriz_features_glcm_CMMD.csv", index=False)
glcm_df_CMMD.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_3,dissimilarity_ang_45_dist_3,dissimilarity_ang_90_dist_3,dissimilarity_ang_135_dist_3,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_3,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_3,energy_ang_45_dist_3,energy_ang_90_dist_3,energy_ang_135_dist_3,pathology
0,0.625476,0.707031,0.627003,0.705654,0.849761,0.852868,0.843262,0.836354,0.991534,0.989223,...,0.841153,0.917298,0.917229,0.917193,0.917241,0.917209,0.917115,0.916896,0.917144,Benign
1,1.184211,1.307271,1.150881,1.317487,1.571586,1.544999,1.489975,1.533504,0.989185,0.986792,...,0.661294,0.813474,0.813478,0.813445,0.813389,0.813267,0.813380,0.813187,0.813200,Benign
2,1.140270,1.302220,1.141987,1.289631,1.560706,1.582450,1.577239,1.528206,0.991724,0.989122,...,0.771217,0.878428,0.878354,0.878307,0.878340,0.878300,0.878218,0.877938,0.878190,Benign
3,1.682375,1.903251,1.648690,1.878657,2.259558,2.274373,2.170288,2.173169,0.987955,0.984806,...,0.596873,0.772685,0.772592,0.772693,0.772697,0.772412,0.772365,0.772438,0.772575,Benign
4,1.659040,1.898933,1.665683,1.884636,2.188467,2.233676,2.251008,2.179951,0.986040,0.981888,...,0.609476,0.781069,0.780941,0.780920,0.780936,0.780836,0.780698,0.780388,0.780689,Benign


### CBIS-DDSM 

In [4]:
name_json = "extracted_metadata_CBIS-DDSM"

list_metadata = load_json(name_json, path_json)
glcm_features = []
labels = []

for metadata in list_metadata:
    metadata_csv = metadata['metadata_csv']
    
    directory = Path(metadata_csv['original_image_path'])
    paths_dicom_file = list(directory.rglob("*.dcm*"))
    
    for path_dicom_file in paths_dicom_file:
        dicom_file = dcmread(path_dicom_file)
        
        labels.append(metadata_csv['pathology'])
        
        # Carrega a matriz da imagem original
        image = dicom_file.pixel_array

        # Redimensiona a imagem original e salva no array
        if image.max() > 256:
            image = (image / 256).astype(np.uint8)

        # image = cv2.equalizeHist(image)
        features = get_glcm_features(image, distances, angles, 256, True, True, properties)
        glcm_features.append(features)

columns = []
angles_labels = get_angles_labels(angles)

for name in properties:
    for distance in distances:
        for ang in angles_labels:
            columns.append(name + "_ang_" + ang + "_dist_" + str(distance))

glcm_df_CBIS_DDSM = pd.DataFrame(glcm_features, columns=columns)
glcm_df_CBIS_DDSM['pathology'] = labels
glcm_df_CBIS_DDSM.to_csv("../../outputs/mamografia/matriz_features_glcm_CBIS-DDSM.csv", index=False)
glcm_df_CBIS_DDSM.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_3,dissimilarity_ang_45_dist_3,dissimilarity_ang_90_dist_3,dissimilarity_ang_135_dist_3,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_3,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_3,energy_ang_45_dist_3,energy_ang_90_dist_3,energy_ang_135_dist_3,pathology
0,0.796989,0.936951,0.853886,0.960574,1.104016,1.098619,1.102612,1.093094,0.998690,0.998135,...,0.474576,0.689123,0.689054,0.689091,0.689045,0.689050,0.688890,0.688736,0.688895,BENIGN
1,1.231098,1.446281,1.325998,1.504659,1.714750,1.686461,1.739642,1.742159,0.998911,0.998513,...,0.265905,0.516184,0.515934,0.516062,0.515914,0.515644,0.515672,0.515621,0.515659,BENIGN
2,0.786669,0.931541,0.833982,0.958347,1.096969,1.116562,1.160822,1.137839,0.998686,0.998076,...,0.481953,0.694554,0.694418,0.694455,0.694378,0.694310,0.694266,0.694058,0.694228,BENIGN_WITHOUT_CALLBACK
3,0.786669,0.931541,0.833982,0.958347,1.096969,1.116562,1.160822,1.137839,0.998686,0.998076,...,0.481953,0.694554,0.694418,0.694455,0.694378,0.694310,0.694266,0.694058,0.694228,BENIGN_WITHOUT_CALLBACK
4,1.147424,1.354507,1.186222,1.370834,1.610495,1.610298,1.576044,1.579974,0.998917,0.998541,...,0.301521,0.549273,0.549286,0.549392,0.549245,0.549113,0.549106,0.549050,0.549109,BENIGN_WITHOUT_CALLBACK


### INBREAST

In [5]:
name_json = "extracted_metadata_INBREAST"

list_metadata = load_json(name_json, path_json)
glcm_features = []
labels = []

for metadata in list_metadata:
    metadata_csv = metadata['metadata_csv']
    
    paths_dicom_file = [metadata_csv['image_path']]
    
    for path_dicom_file in paths_dicom_file:
        dicom_file = dcmread(path_dicom_file)
        
        label = metadata_csv['bi-rads'].replace("4a", "3").replace("4b", "3").replace("4c", "5")  # noqa: E501
        label = int(label)
        
        if label == 0:
            label = 'INCONCLUSIVE'
        elif label == 1:
            label = 'NORMAL'
        elif label < 4:
            label = 'BENIGN'    
        else:
            label = 'MALIGNANT'
            
        labels.append(label)
        
        # Carrega a matriz da imagem original
        image = dicom_file.pixel_array

        # Redimensiona a imagem original e salva no array
        if image.max() > 256:
            image = (image / 256).astype(np.uint8)

        #image = cv2.equalizeHist(image)
        features = get_glcm_features(image, distances, angles, 256, True, True, properties)
        glcm_features.append(features)

columns = []
angles_labels = get_angles_labels(angles)

for name in properties:
    for distance in distances:
        for ang in angles_labels:
            columns.append(name + "_ang_" + ang + "_dist_" + str(distance))

glcm_df_INBREAST = pd.DataFrame(glcm_features, columns=columns)
glcm_df_INBREAST['pathology'] = labels
glcm_df_INBREAST.to_csv("../../outputs/mamografia/matriz_features_glcm_INBREAST.csv", index=False)
glcm_df_INBREAST.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_3,dissimilarity_ang_45_dist_3,dissimilarity_ang_90_dist_3,dissimilarity_ang_135_dist_3,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_3,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_3,energy_ang_45_dist_3,energy_ang_90_dist_3,energy_ang_135_dist_3,pathology
0,0.016037,0.017468,0.015719,0.017442,0.023375,0.022216,0.022405,0.022274,0.997370,0.997112,...,0.781581,0.884444,0.884331,0.884372,0.884333,0.884115,0.884071,0.883897,0.884071,NORMAL
1,0.017885,0.019595,0.017781,0.019532,0.025577,0.024861,0.025191,0.024804,0.997383,0.997089,...,0.744583,0.863376,0.863225,0.863291,0.863231,0.862956,0.862884,0.862705,0.862892,BENIGN
2,0.027353,0.029184,0.026601,0.029546,0.038063,0.035930,0.035288,0.036547,0.996965,0.996790,...,0.657418,0.811537,0.811385,0.811521,0.811337,0.810802,0.810899,0.810777,0.810813,NORMAL
3,0.026346,0.028427,0.025535,0.028125,0.037101,0.035375,0.034291,0.034916,0.996876,0.996582,...,0.664551,0.815822,0.815630,0.815802,0.815675,0.815106,0.815121,0.815061,0.815200,BENIGN
4,0.083185,0.091120,0.083896,0.089706,0.108699,0.111023,0.112860,0.107855,0.994243,0.993665,...,0.302823,0.555955,0.554030,0.555682,0.554339,0.550365,0.549648,0.549058,0.550294,MALIGNANT
